In [11]:
# Step 01: Isolates EKE output from Kial Stewart's mk_KDS75_bottom_EKE[3].py
# script.

# Step 01_1: Regrids botvel onto regular lat/lon from Paul's code provided
# in 17/01/2018 email.

In [12]:
#Import modules
import netCDF4 as nc

In [13]:
#Import EKE (computed by Kial Stewart - script provided in an email)
gridfile = nc.Dataset('/Users/mandithran/Dropbox/PhD/Contourites/eke/KDS75_bottom_terms_for_Paul.nc','r')
xu_ocean = gridfile.variables['xu_ocean'][:] #Change these for finalised
yu_ocean = gridfile.variables['yu_ocean'][:] #Change these for finalised
bot_K = gridfile.variables['bot_K'][:,:] #Change these for finalised
gridfile.close()

#Check shape of arrays
print xu_ocean.shape, yu_ocean.shape, bot_K.shape
#Check min and max of longitudes
print xu_ocean.min(), xu_ocean.max()


(3600,) (2700,) (1, 2700, 3600)
-279.899993896 80.0


In [14]:
# STEP 01 (optional step): Isolate eke and get rid of time variable,
# write file back out (time variable was making it unreadable in Panoply)

#From Kial's script:

eddyfile = nc.Dataset('/Users/mandithran/Dropbox/PhD/Contourites/eke/KDS75_bottom_terms_eke01.nc', 'w', format='NETCDF4')

#For creating x,y dimensions for output netCDF file
xn = len(xu_ocean) #3600 # number of longitude points
yn = len(yu_ocean) #2700 # number of latitude points
eddyfile.createDimension('yu_ocean',yn)
eddyfile.createDimension('xu_ocean',xn)
lat = eddyfile.createVariable('yu_ocean','f8',('yu_ocean',)) #yu_ocean
lat.long_name = 'ucell latitude'
lat.units = 'degrees_N'
lat.cartesian_axis = 'Y'
eddyfile.variables['yu_ocean'][:] = yu_ocean
lon = eddyfile.createVariable('xu_ocean','f8',('xu_ocean',)) #xu_ocean
lon.long_name = 'ucell longitude'
lon.units = 'degrees_E'
lon.cartesian_axis = 'X'
eddyfile.variables['xu_ocean'][:] = xu_ocean
Fssh_bar = eddyfile.createVariable('bot_K','f8',('yu_ocean','xu_ocean',),fill_value=1.e+20)
Fssh_bar.long_name = 'EKE_bottom'
Fssh_bar.units = 'm2s-2'
Fssh_bar.valid_range = -1000., 1000.
Fssh_bar.missing_value = 1.e+20
Fssh_bar.cell_methods = 'time: mean'
Fssh_bar.coordinates = 'lon lat'
Fssh_bar.standard_name = 'EKE_bottom'
eddyfile.variables['bot_K'][:,:] = bot_K
eddyfile.close()


# Note: if you get a "permission denied" error with
# this step, just remove the output file generated 
# at the end of the cell (if present), and it will
# regenerate the file with no problems.

## Step 01_1:

Comments directly from Paul's code:

"This works by over-writing a variable in an existing netcdf file that has geolon_t and geolat_t then use cdo to interpolate to a regular grid. The cdo commands need to be done in the terminal window" (Mandi comment: use magic (!) in iPython notebook)"

Adding a mask helps with computational efficiency while MOM is actually running (time and resources would otherwise be wasted over land). This mask needs to be corrected for before the regridding.


## Potential errors:

If this cell throws a permissions error, stops, or keeps running [*] inexplicably, just restart the kernel and make sure the unmasked_vars_for_cdo_regrid.nc made in this cell has been removed - if you're de-bugging, put this line at the top of the cell: ! rm unmasked_vars_for_cdo_regrid.nc

In [ ]:
import subprocess as sub

# For debugging purposes. Otherwise I get a permissions error (even when running locally). This can probably be permanently rectified by figuring out the source of the permissions error...
! rm unmasked_vars_for_cdo_regrid.nc
! rm unmasked_vars_for_cdo_regrid_01_1.nc


# Import file from Step 01 (previous cell)

# Running locally:
ifile = nc.Dataset(\
                   '/Users/mandithran/Dropbox/PhD/Contourites/eke/KDS75_bottom_terms_eke01.nc',\
                   format='NETCDF4') # local directory

# Raijin path:
#ifile = nc.Dataset(\
#                   '/g/data1/v45/pas561/mom/archive/KDS75/KDS75_bottom_terms_eke01',\
#                   format='NETCDF4')

# Variable name can be queried, or alternatively,
# pop it into Panoply, which will list it. If created
# in previous step, just look it up in previous cell.


#Assign variable
target_var = ifile.variables['bot_K'][:,:]                                    
ifile.close()


# Assign masked and unmasked vars grids as python variables
unmasked_ocean_grid = '/Users/mandithran/Dropbox/PhD/Contourites/eke/unmasked_ocean_grid.nc'
unmasked_vars_for_cdo_regrid = '/Users/mandithran/Dropbox/PhD/Contourites/eke/unmasked_vars_for_cdo_regrid.nc'


# If running locally, you need to rsync the'unmasked_ocean_grid.nc' input file from Raijin.
# Directory: /g/data1/v45/pas561/mom/archive/KDS75/
! ncks -v beta,geolat_t,geolon_t,kmt,geolon_c,geolat_c $unmasked_ocean_grid $unmasked_vars_for_cdo_regrid

ofile = nc.Dataset(unmasked_vars_for_cdo_regrid, 'r')
beta = ofile.variables['beta'][:,:]
ofile.close()


# Check shape - vars must be of the same dimension
print beta.shape
print target_var.shape
beta[:] = target_var # Write variable of interest for regrid


# Overwriting unmasked_vars_for_cdo_regrid.nc file:
ofile = nc.Dataset('/Users/mandithran/Dropbox/PhD/Contourites/eke/unmasked_vars_for_cdo_regrid.nc', 'a')
var = ofile.variables['beta']
print var.shape
print target_var.shape
var[:] = target_var
ofile.sync()
ofile.close()


# Regridding step
! cdo remapbil,r3600x2700 -selname,beta $unmasked_vars_for_cdo_regrid regrid_beta.nc

#For multiple vertical levels?
#cdo remapbil,r3600x2700x75 -selname,temp unmasked_vars_for_cdo_regrid.nc regrid_temp.nc



rm: unmasked_vars_for_cdo_regrid_01_1.nc: No such file or directory
(2700, 3600)
(2700, 3600)
(2700, 3600)
(2700, 3600)
cdo remapbil: Started child process "selname,beta /Users/mandithran/Dropbox/PhD/Contourites/eke/unmasked_vars_for_cdo_regrid.nc (pipe1.1)".
Warning (cdf_set_var) : Inconsistent variable definition for geolat_c!
Warning (cdf_set_var) : Inconsistent variable definition for geolat_t!
Warning (cdf_set_var) : Inconsistent variable definition for geolon_c!
Warning (cdf_set_var) : Inconsistent variable definition for geolon_t!
cdo remapbil: SCRIP bilinear weights from curvilinear (3600x2700) to lonlat (3600x2700) grid, with source mask (6071846)
cdo remapbil:   0%  0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 

## Problems along the way

1. CDO was complaining about an unsupported file type (netCDF4). Apparently, HDF5 had to be configured in a particular way such that it was "netcdf4 supported".
    - Checked cdo config using: cdo -V (saw that packages were not being recognized)
    - Tried uninstalling/reinstalling relevant packages
    - Homebrew suddenly couldn't find cdo
    - Tried exporting file from ipython notebook as netCDF3 classic, but this was futile because my cdo still wasn't working
    - Tried setting up a virtual environment. First time I used a package manager. Then I tried installing all libraries from source, following steps here: tinyurl.com/haaewsk
    - This didn't work (Jasper install instructions were tricky in virtual environment, tried Quick Install instead to no avail tinyurl.com/y7n2h7yu)
    - Found this page tinyurl.com/y9udsqtg which said that homebrew/science tap has now been deprecated - explained why Homebrew wasn't finding formula before
    - SOLUTION uninstall/reinstall cdo using: brew install brewsci/science/cdo;
    or,
    brew upgrade brewsci/science/cdo.
    but if new updates to the formula are needed (on new CDO versions) a new tap must be maintained. See: tinyurl.com/y9udsqtg

2. CDO didn't know what to do with the new file I exported in previous cell before regridding step.
    - Best guess: according to errors, seems like it looks for variables with lat/lon in the name
    - Simplest solution: simply overwrite unmasked_vars_for_cdo_regrid.nc with code provided in email (written in last block of previous cell)
    - Another solution: probably renaming the variables but I can't really be bothered doing that right now.

#Old code for writing out new file:

"""# Write out new grid for cdo regrid step. Originally overwrote unmasked_vars_for_cdo_regrid.nc (see next block).
unmasked_vars_for_cdo_regrid_01_1 = '/Users/mandithran/Dropbox/PhD/Contourites/eke/unmasked_vars_for_cdo_regrid_01_1.nc'
ofile_01_1 = nc.Dataset(unmasked_vars_for_cdo_regrid_01_1, 'w', format='NETCDF4')
xn = len(xu_ocean) #3600 # number of longitude points
yn = len(yu_ocean) #2700 # number of latitude points
ofile_01_1.createDimension('yu_ocean',yn)
ofile_01_1.createDimension('xu_ocean',xn)
lat = ofile_01_1.createVariable('yu_ocean','f8',('yu_ocean',)) #yu_ocean
lat.long_name = 'ucell latitude'
lat.units = 'degrees_N'
lat.cartesian_axis = 'Y'
ofile_01_1.variables['yu_ocean'][:] = yu_ocean
lon = ofile_01_1.createVariable('xu_ocean','f8',('xu_ocean',)) #xu_ocean
lon.long_name = 'ucell longitude'
lon.units = 'degrees_E'
lon.cartesian_axis = 'X'
ofile_01_1.variables['xu_ocean'][:] = xu_ocean
Fssh_bar = ofile_01_1.createVariable('beta','f8',('yu_ocean','xu_ocean',),fill_value=1.e+20)
Fssh_bar.long_name = 'EKE_bottom' # Replace with desired attributes
Fssh_bar.units = 'm2s-2'
Fssh_bar.valid_range = -1000., 1000.
Fssh_bar.missing_value = 1.e+20
Fssh_bar.cell_methods = 'time: mean'
Fssh_bar.coordinates = 'lon lat'
Fssh_bar.standard_name = 'EKE_bottom'
ofile_01_1.variables['beta'][:,:] = beta
ofile_01_1.close()"""